In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    next_states_logits, Qs_logits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits,
                                                                   labels=next_states_labels))
    #################################################### s'', Q' = env(s', a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_next_states_logits, nextQs_logits = Env(actions=next_actions_logits, states=next_states, # given
                                                 hidden_size=hidden_size, action_size=action_size, 
                                                 state_size=state_size, reuse=True)
    #################################################### s'', Q' = env(~s', ~a')
    next_actions_logits_ = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                               reuse=True)
    next_next_states_logits_, nextQs_logits_ = Env(actions=next_actions_logits_, states=next_states_logits, # pred
                                                   hidden_size=hidden_size, action_size=action_size, 
                                                   state_size=state_size, reuse=True)
    ##################################################### GAN
    Qs = tf.reshape(Qs_logits, shape=[-1])
    nextQs = tf.reshape(nextQs_logits, shape=[-1]) * (1-dones)
    nextQs_ = tf.reshape(nextQs_logits_, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=nextQs, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=nextQs, # GAN
                                                                    labels=tf.ones_like(rates))) # max
    ######################################################### Curosity=intrinsic reward
    rloss = tf.square(nextQs-nextQs_)
    sloss = tf.reduce_sum(tf.square(next_next_states_logits-next_next_states_logits_), axis=1)
    #### Q/r loss
    mean, var = tf.nn.moments(x=rloss,axes=[0])
    std = tf.sqrt(var)
    rloss_norm = (rloss - mean)/ std 
    #### State loss
    mean, var = tf.nn.moments(x=sloss,axes=[0])
    std = tf.sqrt(var)
    sloss_norm = (sloss - mean)/ std
    #### Intrinsic_rewards
    intrinsic_rewards = sloss_norm + rloss_norm
    ########################################################## Q = r + 0.99*Q'
    targetQs = rewards + intrinsic_rewards + (gamma * nextQs)
    eloss += tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:0.7207 eloss:6.2395 aloss2:0.8141 exploreP:0.9983
Episode:1 meanR:27.0000 R:37.0000 rate:0.0740 aloss:0.7100 eloss:6.3620 aloss2:0.7838 exploreP:0.9947
Episode:2 meanR:24.0000 R:18.0000 rate:0.0360 aloss:0.7077 eloss:6.1339 aloss2:0.8163 exploreP:0.9929
Episode:3 meanR:22.5000 R:18.0000 rate:0.0360 aloss:0.7098 eloss:6.3934 aloss2:0.7755 exploreP:0.9911
Episode:4 meanR:23.6000 R:28.0000 rate:0.0560 aloss:0.7056 eloss:6.4636 aloss2:0.7612 exploreP:0.9884
Episode:5 meanR:22.0000 R:14.0000 rate:0.0280 aloss:0.7035 eloss:6.4041 aloss2:0.7644 exploreP:0.9870
Episode:6 meanR:21.1429 R:16.0000 rate:0.0320 aloss:0.7030 eloss:6.3502 aloss2:0.7578 exploreP:0.9855
Episode:7 meanR:19.7500 R:10.0000 rate:0.0200 aloss:0.6894 eloss:6.2569 aloss2:0.7541 exploreP:0.9845
Episode:8 meanR:20.0000 R:22.0000 rate:0.0440 aloss:0.7039 eloss:6.3522 aloss2:0.7552 exploreP:0.9823
Episode:9 meanR:20.8000 R:28.0000 rate:0.0560 aloss:0.7014 eloss:6.4840 aloss2:0.7

Episode:80 meanR:21.6914 R:36.0000 rate:0.0720 aloss:0.6980 eloss:5.0133 aloss2:0.6563 exploreP:0.8405
Episode:81 meanR:22.0732 R:53.0000 rate:0.1060 aloss:0.7002 eloss:5.2169 aloss2:0.6593 exploreP:0.8361
Episode:82 meanR:22.0602 R:21.0000 rate:0.0420 aloss:0.7053 eloss:5.0389 aloss2:0.6557 exploreP:0.8344
Episode:83 meanR:22.3214 R:44.0000 rate:0.0880 aloss:0.7019 eloss:4.9794 aloss2:0.6538 exploreP:0.8307
Episode:84 meanR:22.1882 R:11.0000 rate:0.0220 aloss:0.7067 eloss:5.1568 aloss2:0.6570 exploreP:0.8298
Episode:85 meanR:22.1744 R:21.0000 rate:0.0420 aloss:0.7055 eloss:4.9859 aloss2:0.6579 exploreP:0.8281
Episode:86 meanR:22.1609 R:21.0000 rate:0.0420 aloss:0.7162 eloss:5.2389 aloss2:0.6622 exploreP:0.8264
Episode:87 meanR:22.0795 R:15.0000 rate:0.0300 aloss:0.7219 eloss:5.0051 aloss2:0.6704 exploreP:0.8252
Episode:88 meanR:22.0787 R:22.0000 rate:0.0440 aloss:0.6988 eloss:4.8281 aloss2:0.6565 exploreP:0.8234
Episode:89 meanR:21.9556 R:11.0000 rate:0.0220 aloss:0.7000 eloss:4.8974 

Episode:159 meanR:20.6300 R:16.0000 rate:0.0320 aloss:0.7491 eloss:5.2463 aloss2:0.6797 exploreP:0.7161
Episode:160 meanR:20.5700 R:10.0000 rate:0.0200 aloss:0.6859 eloss:4.6032 aloss2:0.6727 exploreP:0.7154
Episode:161 meanR:20.7900 R:33.0000 rate:0.0660 aloss:0.7034 eloss:4.7317 aloss2:0.6703 exploreP:0.7131
Episode:162 meanR:20.8200 R:17.0000 rate:0.0340 aloss:0.7354 eloss:4.9775 aloss2:0.6703 exploreP:0.7119
Episode:163 meanR:20.7500 R:14.0000 rate:0.0280 aloss:0.7082 eloss:4.7522 aloss2:0.6752 exploreP:0.7109
Episode:164 meanR:20.6700 R:19.0000 rate:0.0380 aloss:0.7509 eloss:4.9012 aloss2:0.6782 exploreP:0.7096
Episode:165 meanR:20.4200 R:15.0000 rate:0.0300 aloss:0.7289 eloss:5.0105 aloss2:0.6805 exploreP:0.7085
Episode:166 meanR:20.3800 R:10.0000 rate:0.0200 aloss:0.7272 eloss:4.9448 aloss2:0.6825 exploreP:0.7079
Episode:167 meanR:20.4100 R:16.0000 rate:0.0320 aloss:0.7427 eloss:5.0825 aloss2:0.6808 exploreP:0.7067
Episode:168 meanR:20.4100 R:13.0000 rate:0.0260 aloss:0.7225 elo

Episode:238 meanR:17.1400 R:12.0000 rate:0.0240 aloss:0.7162 eloss:4.7064 aloss2:0.6788 exploreP:0.6267
Episode:239 meanR:17.1300 R:12.0000 rate:0.0240 aloss:0.7279 eloss:4.8964 aloss2:0.6824 exploreP:0.6259
Episode:240 meanR:17.1100 R:17.0000 rate:0.0340 aloss:0.7331 eloss:4.9317 aloss2:0.6845 exploreP:0.6249
Episode:241 meanR:17.1600 R:20.0000 rate:0.0400 aloss:0.7342 eloss:4.8251 aloss2:0.6806 exploreP:0.6236
Episode:242 meanR:17.0700 R:15.0000 rate:0.0300 aloss:0.7268 eloss:4.9556 aloss2:0.6817 exploreP:0.6227
Episode:243 meanR:17.0900 R:15.0000 rate:0.0300 aloss:0.7301 eloss:4.6767 aloss2:0.6796 exploreP:0.6218
Episode:244 meanR:17.0500 R:16.0000 rate:0.0320 aloss:0.7218 eloss:4.8253 aloss2:0.6885 exploreP:0.6208
Episode:245 meanR:17.2300 R:32.0000 rate:0.0640 aloss:0.7210 eloss:4.6950 aloss2:0.6815 exploreP:0.6189
Episode:246 meanR:17.2500 R:25.0000 rate:0.0500 aloss:0.7246 eloss:4.7618 aloss2:0.6855 exploreP:0.6173
Episode:247 meanR:17.2800 R:15.0000 rate:0.0300 aloss:0.7538 elo

Episode:317 meanR:15.3900 R:15.0000 rate:0.0300 aloss:0.7559 eloss:4.8418 aloss2:0.6788 exploreP:0.5569
Episode:318 meanR:15.4200 R:13.0000 rate:0.0260 aloss:0.7769 eloss:4.8763 aloss2:0.6816 exploreP:0.5562
Episode:319 meanR:15.4600 R:17.0000 rate:0.0340 aloss:0.7321 eloss:4.7658 aloss2:0.6834 exploreP:0.5552
Episode:320 meanR:15.4600 R:11.0000 rate:0.0220 aloss:0.7438 eloss:4.8005 aloss2:0.6863 exploreP:0.5546
Episode:321 meanR:15.5400 R:19.0000 rate:0.0380 aloss:0.7069 eloss:4.7233 aloss2:0.6882 exploreP:0.5536
Episode:322 meanR:15.4300 R:11.0000 rate:0.0220 aloss:0.7042 eloss:4.5817 aloss2:0.6821 exploreP:0.5530
Episode:323 meanR:15.4400 R:19.0000 rate:0.0380 aloss:0.7296 eloss:4.7301 aloss2:0.6854 exploreP:0.5520
Episode:324 meanR:15.3900 R:16.0000 rate:0.0320 aloss:0.7549 eloss:4.8934 aloss2:0.6862 exploreP:0.5511
Episode:325 meanR:15.3100 R:10.0000 rate:0.0200 aloss:0.7570 eloss:4.7425 aloss2:0.6838 exploreP:0.5506
Episode:326 meanR:15.3400 R:15.0000 rate:0.0300 aloss:0.7651 elo

Episode:396 meanR:21.4000 R:31.0000 rate:0.0620 aloss:0.6808 eloss:4.7066 aloss2:0.6752 exploreP:0.4654
Episode:397 meanR:21.8100 R:51.0000 rate:0.1020 aloss:0.6904 eloss:4.8868 aloss2:0.6734 exploreP:0.4631
Episode:398 meanR:22.4600 R:75.0000 rate:0.1500 aloss:0.6923 eloss:4.9582 aloss2:0.6731 exploreP:0.4597
Episode:399 meanR:22.8500 R:50.0000 rate:0.1000 aloss:0.6899 eloss:4.9605 aloss2:0.6724 exploreP:0.4574
Episode:400 meanR:23.1100 R:41.0000 rate:0.0820 aloss:0.6904 eloss:4.9893 aloss2:0.6709 exploreP:0.4556
Episode:401 meanR:23.4800 R:48.0000 rate:0.0960 aloss:0.6884 eloss:4.9351 aloss2:0.6707 exploreP:0.4535
Episode:402 meanR:23.6700 R:32.0000 rate:0.0640 aloss:0.7025 eloss:5.1557 aloss2:0.6709 exploreP:0.4520
Episode:403 meanR:24.1000 R:56.0000 rate:0.1120 aloss:0.7080 eloss:5.1356 aloss2:0.6688 exploreP:0.4496
Episode:404 meanR:24.3600 R:38.0000 rate:0.0760 aloss:0.7012 eloss:5.0836 aloss2:0.6713 exploreP:0.4479
Episode:405 meanR:24.8700 R:64.0000 rate:0.1280 aloss:0.6867 elo

Episode:475 meanR:29.1900 R:13.0000 rate:0.0260 aloss:0.7017 eloss:4.4652 aloss2:0.6809 exploreP:0.3827
Episode:476 meanR:29.1500 R:20.0000 rate:0.0400 aloss:0.7221 eloss:4.5601 aloss2:0.6796 exploreP:0.3819
Episode:477 meanR:29.0300 R:13.0000 rate:0.0260 aloss:0.7219 eloss:4.4931 aloss2:0.6794 exploreP:0.3814
Episode:478 meanR:28.7900 R:13.0000 rate:0.0260 aloss:0.7246 eloss:4.7607 aloss2:0.6804 exploreP:0.3810
Episode:479 meanR:28.6600 R:19.0000 rate:0.0380 aloss:0.7475 eloss:4.5185 aloss2:0.6787 exploreP:0.3803
Episode:480 meanR:28.4500 R:14.0000 rate:0.0280 aloss:0.7337 eloss:4.4540 aloss2:0.6803 exploreP:0.3797
Episode:481 meanR:28.2000 R:12.0000 rate:0.0240 aloss:0.7162 eloss:4.4273 aloss2:0.6814 exploreP:0.3793
Episode:482 meanR:28.1700 R:12.0000 rate:0.0240 aloss:0.7083 eloss:4.4209 aloss2:0.6809 exploreP:0.3789
Episode:483 meanR:27.6100 R:11.0000 rate:0.0220 aloss:0.7133 eloss:4.4488 aloss2:0.6809 exploreP:0.3784
Episode:484 meanR:27.1100 R:17.0000 rate:0.0340 aloss:0.7297 elo

Episode:554 meanR:13.4100 R:13.0000 rate:0.0260 aloss:0.7476 eloss:4.5705 aloss2:0.6863 exploreP:0.3474
Episode:555 meanR:13.3300 R:8.0000 rate:0.0160 aloss:0.7061 eloss:4.2521 aloss2:0.6872 exploreP:0.3471
Episode:556 meanR:13.1900 R:10.0000 rate:0.0200 aloss:0.7897 eloss:4.5553 aloss2:0.6802 exploreP:0.3468
Episode:557 meanR:13.1300 R:12.0000 rate:0.0240 aloss:0.7442 eloss:4.5741 aloss2:0.6833 exploreP:0.3464
Episode:558 meanR:13.0800 R:12.0000 rate:0.0240 aloss:0.7554 eloss:4.7695 aloss2:0.6756 exploreP:0.3460
Episode:559 meanR:13.0500 R:12.0000 rate:0.0240 aloss:0.7557 eloss:5.1314 aloss2:0.6696 exploreP:0.3456
Episode:560 meanR:12.9400 R:10.0000 rate:0.0200 aloss:0.8589 eloss:5.4148 aloss2:0.6661 exploreP:0.3452
Episode:561 meanR:12.8500 R:9.0000 rate:0.0180 aloss:0.8046 eloss:5.4245 aloss2:0.6566 exploreP:0.3449
Episode:562 meanR:12.7500 R:11.0000 rate:0.0220 aloss:0.8442 eloss:5.2497 aloss2:0.6358 exploreP:0.3446
Episode:563 meanR:12.7700 R:16.0000 rate:0.0320 aloss:0.8767 eloss

Episode:633 meanR:11.2000 R:11.0000 rate:0.0220 aloss:0.9837 eloss:5.3817 aloss2:0.5613 exploreP:0.3189
Episode:634 meanR:11.2000 R:10.0000 rate:0.0200 aloss:0.9566 eloss:5.3800 aloss2:0.5668 exploreP:0.3186
Episode:635 meanR:11.1800 R:9.0000 rate:0.0180 aloss:1.0986 eloss:5.6556 aloss2:0.5548 exploreP:0.3183
Episode:636 meanR:11.1600 R:11.0000 rate:0.0220 aloss:1.0527 eloss:6.0059 aloss2:0.5897 exploreP:0.3180
Episode:637 meanR:11.1200 R:11.0000 rate:0.0220 aloss:1.0380 eloss:5.5001 aloss2:0.5612 exploreP:0.3176
Episode:638 meanR:11.0900 R:11.0000 rate:0.0220 aloss:1.0897 eloss:5.6584 aloss2:0.5665 exploreP:0.3173
Episode:639 meanR:11.1300 R:13.0000 rate:0.0260 aloss:1.0578 eloss:5.5482 aloss2:0.5700 exploreP:0.3169
Episode:640 meanR:11.1500 R:11.0000 rate:0.0220 aloss:1.0785 eloss:5.5309 aloss2:0.5652 exploreP:0.3166
Episode:641 meanR:11.1300 R:10.0000 rate:0.0200 aloss:0.9964 eloss:5.3897 aloss2:0.5806 exploreP:0.3163
Episode:642 meanR:11.1400 R:11.0000 rate:0.0220 aloss:1.1284 elos

Episode:712 meanR:11.6400 R:10.0000 rate:0.0200 aloss:1.1184 eloss:4.9194 aloss2:0.6938 exploreP:0.2914


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
